In [2]:
import pyvisa
from time import sleep
from matplotlib import pyplot as plt
import numpy as np

In [3]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0x0699::0x03A6::C018103::INSTR', 'ASRL1::INSTR', 'ASRL4::INSTR')


In [13]:
def paramRef(dlc, command):
    dlc.write("(param-ref " + command + ")")
    dlc.read()
    return dlc.read().strip()

def paramDisp(dlc, command):
    dlc.write("(param-disp " + command + ")")
    response = dlc.read()
    while response != 0:
        print(response, end='')
        response=dlc.read()

def exec(dlc, command):
    dlc.write("(exec " + command + ")")

In [23]:
oscilloscope = rm.open_resource(rm.list_resources()[0])
dlc = rm.open_resource(rm.list_resources()[2])
print(oscilloscope.query('*IDN?'))
# print(dlc.write("param-disp 'laser1"))
# print(dlc.read())

dlc.close()
oscilloscope.close()

TEKTRONIX,TDS 2024C,C018103,CF:91.1CT FV:v24.18



Here I'll give examples of other SCPI commands for the tektronix oscilloscope 2024C.
A comprehensive list is available at https://download.tek.com/manual/TBS1000-B-EDU-TDS2000-B-C-TDS1000-B-C-EDU-TDS200-TPS2000-Programmer.pdf

In [22]:
dlc.close()
oscilloscope.close()

In [25]:
def analogConversion(voltage, setPoint=780.000, analogRes=0.001):
    '''
    This function should take an averaged analog output, collected via pyvisa, and convert it into the displayed wavelength on the front panel.
    See manual section 5.2.3
    @param voltage
    @param setpoint: nanometers Inputted into the front of the Burleigh WA1000
    @param analogRes: nanometers Inputted into the front of the Burleigh WA1000
    '''
    # The manual gives an equation $(Analog Output Voltage) =  0.0049*(Deviation in nm)/(Analog Res) + Offset$
    # Offset will need to be calculated via the difference in reading through this program and on the face of the wavemeter

    offset = 0
    deviation = (voltage + offset)/0.0049 * analogRes
    return(setPoint+deviation - 0.0072428680025637959
    )

In [10]:
calibration  = np.array([[780.2328033106735, 780.228],
[780.2443916213673, 780.237],
[780.2164326881837, 780.210],
[780.286704058551, 780.280],
[780.2430637515306, 780.236],
[780.2802851735306, 780.271],
[780.243681134, 780.236],
[780.2795812061837, 780.271]])
np.sum(calibration[:,0]-calibration[:,1])/8

0.0072428680025637959